In [1]:
#Load the necessary libraries.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import os, sys
import itertools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns', None)
warnings.filterwarnings('ignore')

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

Objective = Sales forecast using time series

In [ ]:
df1 = pd.read_csv('../amazon.csv')

df1.info()

In [ ]:
df1.tail()

**Data Wrangling**

In [ ]:
df1.columns

In [ ]:
df1[['rating','rating_count']].info()

In [ ]:
df1.isna().sum()

In [ ]:
# Apply the Data_cleaning script

sys.path.append('../src')
from Data_cleaning import DataHandling

cleaner = DataHandling(df1)

cleaner.apply_cleaning()

df_cleaned = cleaner.get_cleaned_dataframe()

df_cleaned.head()

In [ ]:
#Handle missing values using SimpleImputer
from sklearn.impute import SimpleImputer

Imputer = SimpleImputer(strategy='most_frequent')


df1[['Rating','Rating_count']] = Imputer.fit_transform(df1[['Rating','Rating_count']])

df1[['Rating','Rating_count']].isna().sum()


clean category column, 

In [ ]:
# check missing values
df1.isna().sum()


In [ ]:
# rename columns
new_column_name = {
    'Discounted_price':'Discounted_price_$',
    'Actual_price':'Actual_price_$',
    'Discount_percentage':'Discount_percentage_%',
}

df1.rename(columns=new_column_name, inplace=True)

In [ ]:
df1.info()

In [ ]:
#Cleaning categorical columns

df1.select_dtypes(include='object').nunique()

**Descriptive Statistics**

In [ ]:
num_attributes = df1.select_dtypes(exclude='object')
cat_attributes = df1.select_dtypes(include='object')

**Numerical Attributes**

In [ ]:
# Numerical Attributes
describe = num_attributes.describe().T

describe['range'] = (num_attributes.max() - num_attributes.min()).tolist()
describe['variation coefficient'] = (num_attributes.std() / num_attributes.mean()).tolist()
describe['skew'] = num_attributes.skew().tolist()
describe['kurtosis'] = num_attributes.kurtosis().tolist()

describe

**Categorical Attributes**

In [ ]:
# Categorical aatributes

cat_attributes.describe().T


In [ ]:
df1.columns

In [ ]:
cat_attributes.shape

In [ ]:
#create an only categorical and numerical features dataframe
num_df = df1.select_dtypes(include='float')
cat_df = df1.select_dtypes(include='object')

In [ ]:
num_df[:5]

**Cleaning text or categorical features**

In [ ]:
#categorical dataframe
cat_df[:5]

Category = '|', user_id to review_title = ',' # clean accordingly

In [ ]:
sys.path.append('../src')
from Data_cleaning import CategoricalHandling

cat_cleaner = CategoricalHandling(cat_df)

df2_cleaned = cat_cleaner.apply_cat(lambda x: x.lower() if isinstance(x, str) else x, columns=['User_name'])

df2_cleaned[:5]

In [ ]:
df2_cleaned.shape

In [ ]:

# splits and flatten a column based on the delimiter , or |
def split_and_flatten(column):
    split_values = column.str.split('[|,]', regex=True).to_list()
    flat_list = list(itertools.chain.from_iterable(split_values))
    return flat_list

#Pad the list to the same size/shape
def pad_list(lists):
    max_len = max(len(lst) for lst in lists)
    return [lst + [None] * (max_len - len(lst)) for lst in lists] 

In [ ]:
# global dict to store processed data
flattened_data = {}
#take account of maximum length encountered
max_len = 0

for column in df2_cleaned.columns:
    flattened_column = split_and_flatten(df2_cleaned[column])
    flattened_data[column] = flattened_column
    max_len = max(max_len, len(flattened_column))
    
for key in flattened_data:
    while len(flattened_data[key]) < max_len:
        flattened_data[key].append(None)
    
flattened_data = pd.DataFrame(flattened_data)

for column in flattened_data.columns:
    if flattened_data[column].dtype == object:
        flattened_data[column].fillna('Missing', inplace=True)
    else:
        flattened_data[column].fillna(flattened_data[column].mode()[0], inplace=True)
        
        
flattened_data = flattened_data[~flattened_data.isin(['Missing']).any(axis=1)]



print(flattened_data[:5])

In [ ]:
flattened_data.head()

In [ ]:
flattened_data.shape

In [ ]:
num_df.info()

In [ ]:
cat_df.info()

In [ ]:
#merge them together to create one dataframe by creating a new index.
num_df.reset_index(inplace=True)

flattened_data.reset_index(inplace=True)

df2 = pd.merge(num_df,flattened_data, left_on='index', right_on='index', how='outer')

df2.head()

In [ ]:

df2.info()

In [ ]:
#drop the index column and save the dataframe 
if 'index' in df2.columns:
    df2.drop(columns='index', inplace=True)
    
df2.reset_index(drop=True, inplace=True)

df2.to_csv('../data/amazon.csv', index=False)
print('New Dataframe saved as a csv file...')

**Descriptive Statistical analysis on df2**

In [ ]:
df2.head()

In [ ]:
#sort the dataset by their features

num_attributes = df2.select_dtypes(exclude='object')
cat_attributes = df2.select_dtypes(include='object')


**Numerical Attributes**

In [ ]:
describe = num_attributes.describe().T

describe['range'] = (num_attributes.max() - num_attributes.min()).tolist()
describe['variation coefficient'] = (num_attributes.std() / num_attributes.mean()).tolist()
describe['skew'] = num_attributes.skew().tolist()
describe['kurtosis'] = num_attributes.kurtosis().tolist()


describe

**Categorical Attributes**

In [ ]:
cat_attributes.describe().T

## Business Analytics

In [ ]:
df2.columns

In [ ]:
df2.head()

**Sale analysis**

In [ ]:
# Revenue calculation

#calculate the revenue for each product
df2["Revenue_$"] = df2['Discount_percentage_%'] * df2['Rating_count']

#calculate the revenue for all products
total_revenue = df2['Revenue_$'].sum()

print(f'Total Revenue: ${total_revenue}')

df2.head()

In [ ]:
# Top Product: Best selling product by revenue
top_prod_sorted = df2.sort_values(by='Revenue_$', ascending=False)

Top_product = top_prod_sorted

print(f"Top Best-Selling Product by Revenue:\n {Top_product[['Product_id', 'Product_name', 'Revenue_$']][:6]}") 

In [ ]:
# Top Category: Which Product categories generate the most revenue

#group the category by the sum of revenue
category_revenue = df2.groupby('Category')[['Product_name','Revenue_$']].sum().reset_index()
#sort the category 
category_revenue_sorted = category_revenue.sort_values(by='Revenue_$', ascending=False)
#identify the top categories
top_categories = category_revenue_sorted

print(f"Top Product categories by Revenue: \n {top_categories.head(5)}")


**Customer Analysis**

In [ ]:
from datetime import datetime, timedelta

# create a possible date for purchase date

start_date = datetime(2021,1,1)
end_date = datetime(2023, 12, 31)

def random_date(start, end):
    """Generate random datetime between start and end."""
    return start + timedelta(days=np.random.randint(0, (end - start).days))

df2['Purchase_date'] = df2.apply(lambda x: random_date(start_date, end_date), axis=1)




In [ ]:
# Customer segmentation based on their purchase behaviour

#confirm the datetime preparation
today = datetime.now()
df2['Purchase_date'] = pd.to_datetime(df2['Purchase_date'])

#Calculate Recency, Frequency, and Monetary values for each customer
rfm = df2.groupby('User_id').agg({
    'Purchase_date': lambda x: (today - x.max()).days,
    'User_id': 'count',
    'Discounted_price_$': 'sum'
})

#rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']

rfm = rfm.reset_index()

#rank customers based on RFM values
rfm['R_rank'] = rfm['Recency'].rank(ascending=True)
rfm['F_rank'] = rfm['Frequency'].rank(ascending=False)
rfm['M_rank'] = rfm['Monetary'].rank(ascending=False)

#create a combined RFM score by summing the ranks
rfm['RFM_Score'] = rfm['R_rank'] + rfm['F_rank'] + rfm['M_rank']

#Segment customers based on the RFM score
rfm['Customer_Segment'] = pd.qcut(rfm['RFM_Score'].astype(int), q=4, labels=['Low', 'Medium','High', 'VIP'])

rfm.head(10)

In [ ]:
rfm.to_csv('../data/cleaned/rfm.csv', index=False)

print(rfm.shape)

**Price Analysis**

In [ ]:
import scipy
from scipy.stats import ttest_ind, f_oneway

#calculating additional metric

df2['Discount_amount'] = df2['Actual_price_$'] - df2['Discounted_price_$']
df2['Sales_volume'] = df2['Rating_count'] #using this as proxy for sales volumne

#categorize discount into bins
discount_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
discount_labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
df2['Discount_bin'] = pd.cut(df2['Discount_percentage_%'], bins=discount_bins, labels=discount_labels, include_lowest=True)

#Group the discount bin and calculate average sales volumne
discount_effectiveness = df2.groupby('Discount_bin').agg({
    'Sales_volume':['mean','sum'],
    'Discount_percentage_%':'count'
}).reset_index()

#Rename the columns
discount_effectiveness.columns = ['Discount_bin', 'Avg_Sales_Volume', 'Total_Sales_Volume', 'Count']

# Statistical test to compare sales volumes across discount levels
no_discount_sales = df2[df2['Discount_percentage_%'] == 0]['Sales_volume']
discounted_sales = df2[df2['Discount_percentage_%'] > 0]['Sales_volume']

# T-test for two independent samples
t_stat, p_val = ttest_ind(no_discount_sales, discounted_sales, equal_var=False)
print(f"T-test results: t-statistic = {t_stat}, p-value = {p_val}")

# comparing more than two groups (ANOVA)
anova_results = f_oneway(*[df2[df2['Discount_bin'] == bin]['Sales_volume'] for bin in discount_labels if bin in df2['Discount_bin'].values])
print(f"ANOVA results: F-statistic = {anova_results.statistic}, p-value = {anova_results.pvalue}")

discount_effectiveness.head()

**Price Elasticity**

In [ ]:
#Calculate percentage change in price and quantity demanded
df2['Price_change_%'] = df2['Actual_price_$'].pct_change() * 100
df2['Quantity_change_%'] = df2['Sales_volume'].pct_change() * 100

#Handle NaN values 
df2[['Price_change_%','Quantity_change_%']] = df2[['Price_change_%','Quantity_change_%']].fillna(df2[['Price_change_%','Quantity_change_%']].mean())

#Calculate Price Elasticity of Demand (PED)
df2['PED'] = df2['Quantity_change_%'] / df2['Price_change_%']

print(df2[['Product_id', 'Actual_price_$', 'Sales_volume', 'Price_change_%', 'Quantity_change_%', 'PED']][:5])

In [ ]:
df2.head()

In [ ]:
df2.isna().sum()

In [ ]:
#fix the NaN in PED with the median
df2['PED'] = df2['PED'].fillna(df2['PED'].median())
#check for NaN values
df2.isna().sum()

In [ ]:
# get the selling price

new_column = {
    "Discount_amount":"Selling_price_$"
}

df2.rename(columns=new_column, inplace=True)

In [ ]:
df2.info()

In [ ]:
df2.to_csv('../data/cleaned/amazon.csv', index=False)
print(f'Successfully saved the new dataframe in a csv file format')

df2.head()

## Exploratory Data Analysis (EDA)

**Univariate analysis**

**Bivariate analysis**

Tested Deep Learning and Traditional models
1.   ARIMA – Captures time-dependent patterns and trends.
2.   LSTM – Uses deep learning to capture long-term dependencies in sequential data.
3. Decision Tree Regressor – A baseline traditional model that handles non-sequential data.
4. Random Forest Regressor – Improves on decision trees through ensembling.
5. SVM (Support Vector Machine) – A sensitive model that can handle non-linear relationships, good for small datasets.
6. ANN (Artificial Neural Network) – A powerful but computationally intensive deep learning model.


##ARIMA

In [ ]:
# 1. Convert 'Purchase_date' to datetime and sort the dataset by it
df2['Purchase_date'] = pd.to_datetime(df2['Purchase_date'])
df2 = df2.sort_values('Purchase_date')  # Corrected the typo
df2.set_index('Purchase_date', inplace=True)

# 2. Create time series data for ARIMA
time_series_data = df2['Revenue_$']

# 3. Check stationarity using ADF test


adf_result = adfuller(time_series_data)
print(f"ADF Statistic: {adf_result[0]}")
print(f"p-value: {adf_result[1]}")

if adf_result[1] > 0.05:
    print(f"Time series is non-stationary, Differencing needed")
else:
    print(f"Time series is stationary, no differencing needed")



In [ ]:
from statsmodels.tsa.arima.model import ARIMA


# 1. Define and fit ARIMA model on 'time_series_data' (p=1, d=0, q=1 as a starting point; adjust as needed)
arima_model = ARIMA(time_series_data, order=(1, 0, 1))  # Adjust order as needed
arima_result = arima_model.fit()

# 2. Make predictions on the test set
# Define start and end for test set predictions
train_size = int(len(time_series_data) * 0.4)  # 40% training, 60% testing
start = train_size  # Start of test set
end = len(time_series_data) - 1  # End of dataset

# Generate predictions for the test period
arima_preds = arima_result.predict(start=start, end=end)

# Separate the actual test values for evaluation
y_test_arima = time_series_data[start:]

# 3. Evaluate the ARIMA model
arima_mse = mean_squared_error(y_test_arima, arima_preds)
arima_mae = mean_absolute_error(y_test_arima, arima_preds)
arima_r2 = r2_score(y_test_arima, arima_preds)

print(f'ARIMA - MSE: {arima_mse}, MAE: {arima_mae}, R2: {arima_r2}')

##LSTM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Add time-based features
df2['month'] = df2.index.month
df2['day_of_week'] = df2.index.dayofweek
df2['quarter'] = df2.index.quarter

# Add lagged and rolling average features
df2['Revenue_Lag1'] = df2['Revenue_$'].shift(1)
df2['Revenue_Lag2'] = df2['Revenue_$'].shift(2)
df2['Revenue_7d_avg'] = df2['Revenue_$'].rolling(window=7).mean()
df2['Revenue_30d_avg'] = df2['Revenue_$'].rolling(window=30).mean()



In [ ]:
# Drop any rows with NaN values created by rolling/lagging
df2 = df2.dropna()

# Separate features and target
features = df2.drop(columns=['Revenue_$'])
target = df2['Revenue_$']

# Ensure only numeric features for model input or encode the catergorically variables
features = features.select_dtypes(include=[float, int])

#replace infinity values with NaN, then fill with mean
features = features.replace([np.inf, -np.inf], np.nan)

features = features.fillna(features.mean())

#check for infinity values
print("Infinity values:", np.isinf(features).sum())

#check for very large values
print("max values in each column:")
print(features.max())



In [ ]:
# Split into train and test sets (40% train, 60% test)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.6, shuffle=False)


# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape input for LSTM (samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))



In [ ]:
# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, X_train_scaled.shape[1])))
lstm_model.add(Dense(1))  # Output layer for regression

# Compile and train the model
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=1)



In [ ]:
# Make predictions
lstm_preds = lstm_model.predict(X_test_lstm)
lstm_preds_rescaled = lstm_preds.flatten()



In [ ]:
# LSTM Model Evaluation
lstm_mse = mean_squared_error(y_test, lstm_preds_rescaled)
lstm_mae = mean_absolute_error(y_test, lstm_preds_rescaled)
lstm_r2 = r2_score(y_test, lstm_preds_rescaled)

print(f'LSTM - MSE: {lstm_mse}, MAE: {lstm_mae}, R2: {lstm_r2}')

##Decision Tree

In [ ]:
# Decision Tree

dt_model = DecisionTreeRegressor()
dt_model.fit(X_train_scaled, y_train)
dt_preds = dt_model.predict(X_test_scaled)

dt_mse = mean_squared_error(y_test, dt_preds)
dt_mae = mean_absolute_error(y_test, dt_preds)
dt_r2 = r2_score(y_test, dt_preds)

print(f"Decision Tree - MSE: {dt_mse}, MAE: {dt_mae}, R2: {dt_r2}")



##Random Forest

In [ ]:
# Random Forest

rf_model = RandomForestRegressor()
rf_model.fit(X_train_scaled, y_train)
rf_preds = rf_model.predict(X_test_scaled)

rf_mse = mean_squared_error(y_test, rf_preds)
rf_mae = mean_absolute_error(y_test, rf_preds)
rf_r2 = r2_score(y_test, rf_preds)

print(f"Random Forest - MSE: {rf_mse}, MAE: {rf_mae}, R2: {rf_r2}")



##SVM

In [ ]:
# Support Vector Machine (SVM)

svm_model = SVR()
svm_model.fit(X_train_scaled, y_train)
svm_preds = svm_model.predict(X_test_scaled)

svm_mse = mean_squared_error(y_test, svm_preds)
svm_mae = mean_absolute_error(y_test, svm_preds)
svm_r2 = r2_score(y_test, svm_preds)

print(f"SVM - MSE: {svm_mse}, MAE: {svm_mae}, R2: {svm_r2}")



##ANN

In [ ]:
# Artificial Neural Network (ANN)


ann_model = Sequential()
ann_model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
ann_model.add(Dense(32, activation='relu'))
ann_model.add(Dense(1))  # Output layer for regression

ann_model.compile(optimizer='adam', loss='mse')
ann_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32)

ann_preds = ann_model.predict(X_test_scaled)
ann_preds_rescaled = ann_preds.flatten()

ann_mse = mean_squared_error(y_test, ann_preds_rescaled)
ann_mae = mean_absolute_error(y_test, ann_preds_rescaled)
ann_r2 = r2_score(y_test, ann_preds_rescaled)

print(f"ANN - MSE: {ann_mse}, MAE: {ann_mae}, R2: {ann_r2}")


Best fit models: Decision Tree and 

*   List item
*   List item

Random Forest

---
Reason: R2 closest to value of 1
